In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotion-dataset-deit/emotion-dataset-2/val/angry/PublicTest_21334851.jpg
/kaggle/input/emotion-dataset-deit/emotion-dataset-2/val/angry/PublicTest_66091352.jpg
/kaggle/input/emotion-dataset-deit/emotion-dataset-2/val/angry/PrivateTest_96606536.jpg
/kaggle/input/emotion-dataset-deit/emotion-dataset-2/val/angry/PrivateTest_69720002.jpg
/kaggle/input/emotion-dataset-deit/emotion-dataset-2/val/angry/PublicTest_90378794.jpg
/kaggle/input/emotion-dataset-deit/emotion-dataset-2/val/angry/PublicTest_49898176.jpg
/kaggle/input/emotion-dataset-deit/emotion-dataset-2/val/angry/PrivateTest_88815839.jpg
/kaggle/input/emotion-dataset-deit/emotion-dataset-2/val/angry/PublicTest_92063997.jpg
/kaggle/input/emotion-dataset-deit/emotion-dataset-2/val/angry/PrivateTest_91554872.jpg
/kaggle/input/emotion-dataset-deit/emotion-dataset-2/val/angry/PrivateTest_62257937.jpg
/kaggle/input/emotion-dataset-deit/emotion-dataset-2/val/angry/PublicTest_17026620.jpg
/kaggle/input/emotion-dataset-deit/emo

In [2]:
from datasets import load_dataset

dataset = load_dataset("imagefolder",
                       data_dir="/kaggle/input/emotion-dataset-deit/emotion-dataset-2")

labels = dataset["train"].features["label"].names

label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for idx, label in enumerate(labels)}

Resolving data files:   0%|          | 0/2015 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/407 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/139 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
label2id

{'angry': 0, 'happy': 1, 'neutral': 2, 'sad': 3}

In [4]:
#from transformers import DeiTImageProcessor, DeiTForImageClassification
from transformers import AutoFeatureExtractor, ViTForImageClassification # for tiny model
model_path = "facebook/deit-tiny-patch16-224" #tiny model
#model_path = "facebook/deit-small-distilled-patch16-224"


processor = AutoFeatureExtractor.from_pretrained(model_path, return_tensors = "pt")

mymodel =  ViTForImageClassification.from_pretrained(model_path,
                                                  id2label = id2label,
                                                  label2id = label2id,
                                                  ignore_mismatched_sizes = True)



'''
processor = DeiTImageProcessor.from_pretrained(model_path, return_tensors = "pt")

mymodel = DeiTForImageClassification.from_pretrained(model_path,
                                                  id2label = id2label,
                                                  label2id = label2id,
                                                  ignore_mismatched_sizes = True)
'''




import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mymodel.to(device)  # Send the model to the available device
print(f"Model is running on: {device}")

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/23.0M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at facebook/deit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([4, 192]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model is running on: cuda


In [5]:
from torchvision.transforms import v2

# Augmentation for training
augmenting = v2.Compose([
    v2.RandomHorizontalFlip(0.4),
    v2.RandomVerticalFlip(0.2),
])

# Batched preprocessing for training
def train_transforming(examples):
    # Apply augmentations and process images
    images = [augmenting(image.convert("RGB")) for image in examples["image"]]
    pixel_values = processor(images, return_tensors="pt")['pixel_values']
    return {"pixel_values": pixel_values, "label": examples["label"]}

# Batched preprocessing for validation/testing
def test_transforming(examples):
    # Process images without augmentations
    images = [image.convert("RGB") for image in examples["image"]]
    pixel_values = processor(images, return_tensors="pt")['pixel_values']
    return {"pixel_values": pixel_values, "label": examples["label"]}

# Apply transformations
dataset["train"] = dataset["train"].map(train_transforming, batched=True)
dataset["validation"] = dataset["validation"].map(test_transforming, batched=True)

# Print to verify
#print(dataset["train"][0])  # Check the first sample


Map:   0%|          | 0/2015 [00:00<?, ? examples/s]

Map:   0%|          | 0/407 [00:00<?, ? examples/s]

In [6]:
def collate_fn(all_image):
    pixel_values = torch.stack([
        single_image["pixel_values"] if isinstance(single_image["pixel_values"], torch.Tensor) else torch.tensor(single_image["pixel_values"])
        for single_image in all_image
    ])
    
    labels = torch.tensor([single_image["label"] for single_image in all_image])
    
    return {"pixel_values": pixel_values, "labels": labels}



from sklearn.metrics import accuracy_score
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Ensure predictions and labels are numpy arrays
    predictions = np.argmax(predictions, axis=1)  # Get the index of the highest probability
    if isinstance(labels, np.ndarray):  # Check if labels are already a NumPy array
        labels = labels
    else:  # Convert labels to NumPy if they're tensors
        labels = labels.numpy()
        
    return dict(accuracy=accuracy_score(predictions, labels))


In [7]:
from transformers import TrainingArguments

num_of_epochs = 50

training_args = TrainingArguments(
    output_dir = "/kaggle/working/",
    evaluation_strategy = "steps",
    per_device_train_batch_size =32,
    per_device_eval_batch_size = 32,
    learning_rate = 2e-5,
    weight_decay = 0.01,
    logging_strategy = "steps",
    logging_steps=250,
    save_total_limit = 1,
    metric_for_best_model = "accuracy",
    load_best_model_at_end = True,
    greater_is_better = True,
    logging_dir='./logs',
    num_train_epochs = num_of_epochs,
    fp16=True,  # Mixed precision for faster training (recommended on T4 GPUs)
    ddp_find_unused_parameters=False,  # Avoid errors in distributed mode
)


from transformers import Trainer

trainer = Trainer(
    model = mymodel,
    args = training_args,
    train_dataset=dataset["train"],
    eval_dataset = dataset["validation"],
    data_collator= collate_fn,
    compute_metrics = compute_metrics,
    tokenizer = processor,
    #callbacks = [log_callback]
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [8]:
os.environ["WANDB_DISABLED"] = "true"

trainer.train()

trainer.save_model("/kaggle/working/fb-deit_tiny_best.pt")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
250,0.907900,1.253924,0.486486
500,0.136700,2.174147,0.484029
750,0.004600,3.499330,0.484029
1000,0.000100,4.806877,0.484029
1250,0.000000,5.434900,0.486486
1500,0.000000,5.622721,0.488943


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

In [7]:
!pip show transformers

Name: transformers
Version: 4.45.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: kaggle-environments


In [2]:
!numpy --version

/bin/bash: line 1: numpy: command not found
